In [1]:
#Import dependencies
import pandas as pd
import geopandas as gpd
from pandas_geojson import to_geojson
from shapely.geometry import Point
import json

C:\Users\ocean\AppData\Local\Temp\ipykernel_36932\273604607.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Create path to the CSV
path = './Resources/US_Lightning_Forest_Fires.csv'
csv = pd.read_csv(path)
#Create DataFrame
df = pd.DataFrame(csv)

In [3]:
#Drop columns we don't need
columns_to_drop = ['Unnamed: 0', 'DISCOVERY_TIME', 'FIPS_CODE', 'FIPS_NAME', 'Days_to_extinguish_fire', 'FIPS_CODE', 'FIPS_NAME']
df2 = df.drop(columns=columns_to_drop)
#Drop null values
df2.dropna(subset=['CONT_TIME'])
#Convert "Fire Date" to datetime
df2['Fire_Date'] = pd.to_datetime(df['Fire_Date'], format='%m-%d-%Y', errors='coerce')

In [4]:
#Save new CSV file
csv_path = './Resources/Wildfires_2.csv'
df2.to_csv(csv_path, index = False)

In [5]:
#Create path and read in new CSV
path_2 = './Resources/Wildfires_2.csv'
csv_2 = pd.read_csv(path_2)
#Convert CSV into DF
df_2 = pd.DataFrame(csv_2)

In [6]:
#Create different DataFrame's for each of the time periods we decided to visualize
first_years_df = df_2[df_2["FIRE_YEAR"].between(2000, 2004)]
second_years_df = df_2[df_2["FIRE_YEAR"].between(2005, 2009)]
third_years_df = df_2[df_2["FIRE_YEAR"].between(2010, 2015)]

In [7]:
#Define a function to convert the DataFrames into GeoJson files

def create_geojson(df, filename):
    # Store points for coordinates in a variable
    geometry = [Point(xy) for xy in zip(df['LONGITUDE'], df['LATITUDE'])]

    # Create GeoDataFrame and add geometry column
    geo_df = gpd.GeoDataFrame(df, geometry=geometry)

    # Drop unnecessary columns
    gdf = geo_df.drop(['LATITUDE', 'LONGITUDE', 'index',
                      'STAT_CAUSE_DESCR', 'Fire_Date'], axis=1).copy()
    gdf = gdf.dropna()

    # Dropping unnecessary size classes
    gdf = gdf[~gdf['FIRE_SIZE_CLASS'].isin(['A', 'B', 'C'])]

    # Sort Values
    gdf.sort_values('FIRE_YEAR', inplace=True)

    # Convert non-spatial columns to properties
    properties = gdf.drop(['geometry'], axis=1).to_dict(orient='records')

    # Create a list for GeoJSON-compatible geometries
    geojson_geometries = gdf['geometry'].apply(lambda point: {'type': 'Point', 'coordinates': (
        point.x, point.y)} if point.is_valid else None).tolist()

    # Add properties to GeoJSON
    geojson_data = {
        "type": "FeatureCollection",
        "features": []
    }

    for feature, prop in zip(geojson_geometries, properties):
        # Check if the geometry is valid
        if feature is not None and 'coordinates' in feature and len(feature['coordinates']) == 2:
            geojson_data['features'].append({
                "type": "Feature",
                "geometry": feature,
                "properties": prop
            })

    # Save GeoJSON to a file
    with open(filename, 'w') as f:
        json.dump(geojson_data, f, indent=2)


# Call the function for each DataFrame
create_geojson(first_years_df, './Resources/first_years.geojson')
create_geojson(second_years_df, './Resources/second_years.geojson')
create_geojson(third_years_df, './Resources/third_years.geojson')
